In [ ]:
import numpy as np

# =========================
# Dense Layer
# =========================

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, weights, biases):
        self.weights = np.array(weights, dtype=float)
        self.biases = np.array(biases, dtype=float)
        self.output = None
        self.inputs = None
        self.dweights = None
        self.dbiases = None

    def forward(self, inputs):
        self.inputs = np.array(inputs, dtype=float)
        self.output = np.dot(self.inputs, self.weights) + self.biases

    def backward(self, dvalues, learning_rate=0.1):
        # Gradients
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Update weights & biases
        self.weights -= learning_rate * self.dweights
        self.biases -= learning_rate * self.dbiases


# =========================
# ReLU Activation
# =========================
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


# =========================
# Mean Squared Error Loss
# =========================
class Loss_MSE:
    def forward(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2)

    def backward(self, y_pred, y_true):
        samples = len(y_pred)
        self.dinputs = -2 * (y_true - y_pred) / samples


# =========================
# MAIN (Training Loop)
# =========================
np.set_printoptions(precision=3, suppress=True)

# Input (single sample, 2 features)
X = [[2, 3]]

# Target output
y_true = [[10, 5]]

# Define a dense layer (2 inputs → 2 outputs)
layer1 = Layer_Dense(
    n_inputs=2, n_neurons=2,
    weights=[[1, 2],
             [3, 4]],
    biases=[[1, 1]]
)

activation1 = Activation_ReLU()
loss_fn = Loss_MSE()

# Training loop
epochs = 15
for step in range(1, epochs+1):
    # ---- Forward pass ----
    layer1.forward(X)
    activation1.forward(layer1.output)

    # ---- Loss ----
    loss = loss_fn.forward(activation1.output, y_true)

    # Print details
    print(f"\n===== Step {step} =====")
    print(f"Output: {activation1.output}")
    print(f"Loss: {loss:.4f}")
    print(f"Weights:\n{layer1.weights}")
    print(f"Biases:\n{layer1.biases}")

    # ---- Backward pass ----
    loss_fn.backward(activation1.output, y_true)
    activation1.backward(loss_fn.dinputs)
    layer1.backward(activation1.dinputs, learning_rate=0.01)
    



===== Step 1 =====
Output: [[12. 17.]]
Loss: 74.0000
Weights:
[[1. 2.]
 [3. 4.]]
Biases:
[[1. 1.]]

===== Step 2 =====
Output: [[11.44 13.64]]
Loss: 38.3616
Weights:
[[0.92 1.52]
 [2.88 3.28]]
Biases:
[[0.96 0.76]]

===== Step 3 =====
Output: [[11.037 11.221]]
Loss: 19.8867
Weights:
[[0.862 1.174]
 [2.794 2.762]]
Biases:
[[0.931 0.587]]

===== Step 4 =====
Output: [[10.746  9.479]]
Loss: 10.3092
Weights:
[[0.821 0.926]
 [2.731 2.388]]
Biases:
[[0.91  0.463]]

===== Step 5 =====
Output: [[10.537  8.225]]
Loss: 5.3443
Weights:
[[0.791 0.746]
 [2.687 2.12 ]]
Biases:
[[0.896 0.373]]

===== Step 6 =====
Output: [[10.387  7.322]]
Loss: 2.7705
Weights:
[[0.77  0.617]
 [2.654 1.926]]
Biases:
[[0.885 0.309]]

===== Step 7 =====
Output: [[10.279  6.672]]
Loss: 1.4362
Weights:
[[0.754 0.525]
 [2.631 1.787]]
Biases:
[[0.877 0.262]]

===== Step 8 =====
Output: [[10.201  6.204]]
Loss: 0.7445
Weights:
[[0.743 0.458]
 [2.614 1.687]]
Biases:
[[0.871 0.229]]

===== Step 9 =====
Output: [[10.144  5.867]